# Imports & Settings


In [1]:
# Imports
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from deep_translator import GoogleTranslator
import re 
from math import isnan

pd.set_option("display.max_columns", None)
pd.set_option("display.max_colwidth", None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_seq_items', None)

In [2]:
# dtype={'type': str} prevents being confused with data type for large data sets
train = pd.read_csv('data/train.csv', index_col='id', dtype={'type': str})
test = pd.read_csv('data/test.csv', index_col='id', dtype={'type': str})
train_translated = pd.read_csv('data/train_translated.csv', dtype={'type': str})
test_translated = pd.read_csv('data/test_translated.csv', index_col='id', dtype={'type': str})
combined_data = pd.read_csv('data/combined_data.csv', index_col='id', dtype={'type': str})
combined_data_translated = pd.read_csv('data/combined_data_translated.csv', index_col='id', dtype={'type': str})
combined_data_fully_translated = pd.read_csv('data/combined_data_fully_translated.csv', index_col='id', dtype={'type': str})
prep = pd.read_csv('data/prep.csv', index_col='id', dtype={'type': str})

/tmp/ipykernel_155859/1699356385.py:9: DtypeWarning: Columns (226,235) have mixed types. Specify dtype option on import or set low_memory=False.
  prep = pd.read_csv('data/prep.csv', index_col='id', dtype={'type': str})


# Delete unneeded features


In [9]:
data = prep.copy()

In [13]:
data.drop(columns=['full_nr', 'name','commentary','country_and_unit','text','parameter','unit','value','legend','initial_info','additional_text'], inplace=True)

In [15]:
data.to_csv('data/prep.csv')

In [3]:
data = prep.copy()

In [5]:
data.drop(columns=['text_features'])

: 

: 

In [ ]:
data.to_csv('data/prep.csv')

# resplit test/train


In [ ]:
data = prep.copy()

In [ ]:
data.to_csv('data/prep.csv')